## Import libraries

In [1]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.pyplot import axes, title
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn import metrics
import seaborn as sns

## Load the data

In [2]:
# Load the data set for the fourth IoT node
benign_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.benign.csv")
#g_scan_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.scan.csv")
#g_junk_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.junk.csv")
#g_udp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.udp.csv")
#g_tcp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.tcp.csv")
g_combo_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.gafgyt.combo.csv")
#m_scan_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.scan.csv")
#m_ack_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.ack.csv")
#m_syn_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.syn.csv")
#m_udp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.udp.csv")
#m_p_udp_4 = pd.read_csv("C:/Users/gustavo/Documents/Jupyter_notebook/archive/4.mirai.udpplain.csv")

In [5]:
print
print('Shape==>', 'Benign shape: ', benign_4.shape, '//','Bashlite combo: ', g_combo_4.shape)

Shape==> Benign shape:  (175240, 115) // Bashlite combo:  (58152, 115)


## Prepare the data

In [6]:
benign=benign_4.sample(frac=0.25/10,replace=False)
g_combo=g_combo_4.sample(frac=0.25/10,replace=False)
#g_junk=g_junk_4.sample(frac=0.5,replace=False)
#g_scan=g_scan_4.sample(frac=0.5,replace=False)
#g_tcp=g_tcp_4.sample(frac=0.15,replace=False)
#g_udp=g_udp_4.sample(frac=0.15,replace=False)
#m_ack=m_ack_4.sample(frac=0.25,replace=False)
#m_scan=m_scan_4.sample(frac=0.15,replace=False)
#m_syn=m_syn_4.sample(frac=0.25,replace=False)
#m_udp=m_udp_4.sample(frac=0.1,replace=False)
#m_p_udp=m_p_udp_4.sample(frac=0.27,replace=False)


benign['type']='benign'
g_combo['type']='gafgyt_combo'
#g_junk['type']='gafgyt_junk'
#g_scan['type']='gafgyt_scan'
#g_tcp['type']='gafgyt_tcp'
#g_udp['type']='gafgyt_udp'
#m_udp['type']='mirai_udp'
#m_ack['type']='mirai_ack'
#m_scan['type']='mirai_scan'
#m_syn['type']='mirai_syn'
#m_p_udp['type']='mirai_udpplain'

In [7]:
#data=pd.concat([benign,m_udp,g_combo,g_junk,g_scan,g_tcp,g_udp,m_ack,m_scan,m_syn,m_p_udp],
#               axis=0, sort=False, ignore_index=True)

In [8]:
data=pd.concat([benign,g_combo],
               axis=0, sort=False, ignore_index=True)

In [9]:
#how many instances of each class
data.groupby('type')['type'].count()

type
benign          4381
gafgyt_combo    1454
Name: type, dtype: int64

In [10]:
data.shape

(5835, 116)

In [11]:
#shuffle rows of dataframe 
sampler=np.random.permutation(len(data))
data=data.take(sampler)
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,type
4138,1.027446,66.000000,1.820000e-12,1.115699,66.000000,1.820000e-12,1.556357,65.999873,0.000765,8.967985,...,-3.430000e-11,-9.720000e-07,18.943406,66.032367,0.622382,115.570110,235.165546,-0.143558,-0.015041,benign
3150,2.605539,66.000000,3.740000e-07,2.752804,66.000036,3.285049e-03,2.998668,66.311363,27.925317,11.083861,...,9.754959e-02,6.481874e-02,18.754784,75.082472,39.955755,199.554845,104749.165800,5234.235092,0.404784,benign
3604,1.000000,66.000000,7.960000e-07,1.000138,65.999847,9.181350e-04,1.129485,65.844175,0.924626,9.935719,...,-2.356722e-03,-1.808958e-03,19.955713,66.279887,1.811161,115.444590,241.829483,-0.156802,-0.005568,benign
5121,156.451818,74.013023,3.123921e-01,258.694670,74.023563,5.649655e-01,787.047511,74.036392,0.892418,7209.302500,...,0.000000e+00,0.000000e+00,1.678144,74.000000,0.000000,95.268043,0.000000,0.000000,0.000000,gafgyt_combo
906,1.000002,66.000017,2.670770e-04,1.000571,66.004120,6.590660e-02,1.137544,66.933693,17.451074,8.959822,...,-8.590156e+01,-9.209311e-02,20.267631,91.254221,40.441351,123.366006,1738.047173,33.322912,0.033976,benign


In [12]:
#dummy encode labels, store separately
labels_full=pd.get_dummies(data['type'], prefix='type')
labels_full.head()

,type_benign,type_gafgyt_combo
4138,1,0
3150,1,0
3604,1,0
5121,0,1
906,1,0


In [13]:
#drop labels from training dataset
data=data.drop(columns='type')
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
4138,1.027446,66.000000,1.820000e-12,1.115699,66.000000,1.820000e-12,1.556357,65.999873,0.000765,8.967985,...,152.033473,-3.430000e-11,-9.720000e-07,18.943406,66.032367,0.622382,115.570110,235.165546,-0.143558,-0.015041
3150,2.605539,66.000000,3.740000e-07,2.752804,66.000036,3.285049e-03,2.998668,66.311363,27.925317,11.083861,...,54.779015,9.754959e-02,6.481874e-02,18.754784,75.082472,39.955755,199.554845,104749.165800,5234.235092,0.404784
3604,1.000000,66.000000,7.960000e-07,1.000138,65.999847,9.181350e-04,1.129485,65.844175,0.924626,9.935719,...,249.340633,-2.356722e-03,-1.808958e-03,19.955713,66.279887,1.811161,115.444590,241.829483,-0.156802,-0.005568
5121,156.451818,74.013023,3.123921e-01,258.694670,74.023563,5.649655e-01,787.047511,74.036392,0.892418,7209.302500,...,0.000000,0.000000e+00,0.000000e+00,1.678144,74.000000,0.000000,95.268043,0.000000,0.000000,0.000000
906,1.000002,66.000017,2.670770e-04,1.000571,66.004120,6.590660e-02,1.137544,66.933693,17.451074,8.959822,...,1538.457732,-8.590156e+01,-9.209311e-02,20.267631,91.254221,40.441351,123.366006,1738.047173,33.322912,0.033976


In [14]:
#standardize numerical columns
def standardize(df,col):
    df[col]= (df[col]-df[col].mean())/df[col].std()

data_st=data.copy()
for i in (data_st.iloc[:,:-1].columns):
    standardize (data_st,i)

data_st.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc
4138,-0.582053,-0.173257,-0.092066,-0.579735,-0.176363,-0.093959,-0.575224,-0.179290,-0.101285,-0.566920,...,-0.137643,-0.035496,-0.022857,0.163982,-0.163971,-0.302367,-0.071368,-0.189476,-0.039054,-0.015041
3150,-0.559460,-0.173257,-0.092066,-0.565453,-0.176362,-0.093959,-0.570961,-0.175383,-0.100377,-0.566225,...,-0.139795,-0.035474,0.452163,0.155841,-0.040352,0.700361,0.663434,1.888751,0.893323,0.404784
3604,-0.582446,-0.173257,-0.092066,-0.580743,-0.176364,-0.093959,-0.576485,-0.181242,-0.101255,-0.566602,...,-0.135489,-0.035496,-0.036106,0.207669,-0.160590,-0.272061,-0.072466,-0.189343,-0.039056,-0.005568
5121,1.643104,-0.084766,-0.092056,1.667344,-0.083982,-0.093941,1.746222,-0.078500,-0.101256,1.799246,...,-0.141008,-0.035496,-0.022850,-0.581115,-0.055138,-0.318233,-0.248995,-0.194152,-0.039028,0.000000
906,-0.582446,-0.173257,-0.092066,-0.580740,-0.176315,-0.093957,-0.576461,-0.167578,-0.100717,-0.566923,...,-0.106958,-0.054450,-0.697737,0.221130,0.180544,0.712741,-0.003160,-0.159592,-0.033093,0.033976


# Different models

## DBSCAN - Density-Based Spatial Clustering of Applications with Noise

In [15]:
# Compute DBSCAN
db = DBSCAN(eps=4, min_samples=1300).fit(data_st) # variar eps e min_samples para encontrar n clusters desejado (use 13000)
db.labels_

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

for x in range(10, 0, -1):
    eps = 1/(11-x)
    samples = 130*x
    db = DBSCAN(eps=eps, min_samples=samples).fit(data_st)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    
    print('EPS: ', eps, 'Min samples: ', samples)
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    


In [19]:
classe1 = 0
classe2 = 0
outlier = 0
correct = 0
uncorrect = 0
outlier2 = 0
for i in range(len(data_st)):
    if(db.labels_[i] == 0):
        classe1+=1
    elif(db.labels_[i] == 1):
        classe2+=1
    elif(db.labels_[i] == -1):
        outlier+=1
    
    if((db.labels_[i] == 0 and labels_full.type_benign[data_st.index[i]] == 1) or
       (db.labels_[i] == 1 and labels_full.type_gafgyt_combo[data_st.index[i]] == 1)):
        correct+=1
    elif((db.labels_[i] == 0 and labels_full.type_benign[data_st.index[i]] == 0) or
       (db.labels_[i] == 1 and labels_full.type_gafgyt_combo[data_st.index[i]] == 0)):
        uncorrect+=1
    else:
        outlier2+=1
    
    
    
    #dbscan_label = db.labels_[i]
print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', classe1, ' ', 'Malicious: ', classe2, ' ', 'Outlier: ', outlier)
print('Correct: ', correct,' ', 'Uncorrect: ', uncorrect,' ', 'Outliers: ', outlier2)

print('Accuracy: (%)', 'Benign data: ', classe1/len(benign), 'Malicious data: ', classe2/len(g_combo))
print('Complete data: ', correct/len(data))

Benign length:  4381   Gafgyt length:  1454

Results: 
Grouped as:  Benign:  4011   Malicious:  1445   Outlier:  379
Correct:  5456   Uncorrect:  0   Outliers:  379
Accuracy: (%) Benign data:  0.9155443962565625 Malicious data:  0.9938101788170564
Complete data:  0.9350471293916024


## Isolation Forests (IF)

In [20]:
#from sklearn.ensemble import IsolationForest
#clf = IsolationForest(random_state=0).fit(data_st)
#prediction = clf.predict(data_st) # -1 => outlier

## Local Outlier Factor (LOF)

In [21]:
clf = LocalOutlierFactor(n_neighbors=20, algorithm='auto', leaf_size=30, 
                         metric='minkowski', p=2, metric_params=None, 
                         contamination='auto', novelty=False, n_jobs=None)
result = clf.fit_predict(data_st)
result

array([ 1,  1,  1, ...,  1, -1,  1])

In [22]:
normal = 0
malicious = 0
TP = 0; FP = 0; TN = 0; FN = 0

for i in range(len(data_st)):
    if(result[i] == 1):
        normal+=1
    elif(result[i] == -1):
        malicious+=1

    
    if(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 1):
        TP+=1
    elif(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 0):
        FN+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 1):
        FP+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 0):
        TN+=1
        

accuracy = (TP + TN)/(TP + TN + FP + FN)        
precision = TP/(TP + FP)
recall = TP/(TP+FN)
fscore = 2*precision*recall/(precision+recall)


print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', normal, ' ', 'Malicious: ', malicious)

print('Confusion matrix: ', '\tTP:', TP, 'FP:', FP, '\n', '\t\t\t', 'FN:', FN, 'TN:', TN)
print('Accuracy (%): ', accuracy, '\nPrecision: ', precision, '\nRecall: ', recall, '\nF-score: ', fscore)


print('\nLegend:','\nTP: True Positive (Benign data classified as benign)', '\nFP: False Positive (Benign data classified as malicious)')
print('\nFN: False Negative (Malicious data classified as benign)', '\nTN: True Negative (Malicious data classified as malicious)')


Benign length:  4381   Gafgyt length:  1454

Results: 
Grouped as:  Benign:  5007   Malicious:  828
Confusion matrix:  	TP: 3604 FP: 777 
 			 FN: 1403 TN: 51
Accuracy (%):  0.6263924592973437 
Precision:  0.8226432321387811 
Recall:  0.71979229079289 
F-score:  0.7677886663826161


## Elliptic Envelope (EE)

In [23]:
cov = EllipticEnvelope(store_precision=True, assume_centered=False, support_fraction=None,
                       contamination=0.1, random_state=None).fit(data_st) 
# maybe it is better to use Benign to fit the data and then use data_st to predict
# predict returns 1 for an inlier and -1 for an outlier
result = cov.predict(data_st)


C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-2841.226921811231932 > -2857.485660158531118). You may want to try with a higher value of support_fraction (current value: 0.511).
  warnings.warn("Determinant has increased; this should not happen: "
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-2624.603263394645182 > -2633.813144774539523). You may want to try with a higher value of support_fraction (current value: 0.511).
  warnings.warn("Determinant has increased; this sho

In [24]:
normal = 0
malicious = 0
TP = 0; FP = 0; TN = 0; FN = 0

for i in range(len(data_st)):
    if(result[i] == 1):
        normal+=1
    elif(result[i] == -1):
        malicious+=1

    
    if(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 1):
        TP+=1
    elif(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 0):
        FN+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 1):
        FP+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 0):
        TN+=1
        

accuracy = (TP + TN)/(TP + TN + FP + FN)        
precision = TP/(TP + FP)
recall = TP/(TP+FN)
fscore = 2*precision*recall/(precision+recall)


print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', normal, ' ', 'Malicious: ', malicious)

print('Confusion matrix: ', '\tTP:', TP, 'FP:', FP, '\n', '\t\t\t', 'FN:', FN, 'TN:', TN)
print('Accuracy (%): ', accuracy, '\nPrecision: ', precision, '\nRecall: ', recall, '\nF-score: ', fscore)


print('\nLegend:','\nTP: True Positive (Benign data classified as benign)', '\nFP: False Positive (Benign data classified as malicious)')
print('\nFN: False Negative (Malicious data classified as benign)', '\nTN: True Negative (Malicious data classified as malicious)')


Benign length:  4381   Gafgyt length:  1454

Results: 
Grouped as:  Benign:  5251   Malicious:  584
Confusion matrix:  	TP: 3798 FP: 583 
 			 FN: 1453 TN: 1
Accuracy (%):  0.6510711225364182 
Precision:  0.8669253595069619 
Recall:  0.7232908017520472 
F-score:  0.7886212624584718


## One-Class Support Vector Machine (1CSVM)

In [33]:
clf = OneClassSVM(kernel='rbf', degree=3, gamma='auto',
                  coef0=0.0, tol=0.001, nu=0.35, shrinking=True,
                  cache_size=200, verbose=False, max_iter=- 1).fit(data_st)
result = clf.predict(data_st)

#clf.score_samples(X)

In [38]:
normal = 0
malicious = 0
TP = 0; FP = 0; TN = 0; FN = 0

for i in range(len(data_st)):
    if(result[i] == 1):
        normal+=1
    elif(result[i] == -1):
        malicious+=1

    
    if(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 1):
        TP+=1
    elif(result[i] == 1 and labels_full.type_benign[data_st.index[i]] == 0):
        FN+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 1):
        FP+=1
    elif(result[i] == -1 and labels_full.type_benign[data_st.index[i]] == 0):
        TN+=1
        

accuracy = (TP + TN)/(TP + TN + FP + FN)        
precision = TP/(TP + FP)
recall = TP/(TP+FN)
fscore = 2*precision*recall/(precision+recall)


print('Benign length: ', len(benign), ' ', 'Gafgyt length: ', len(g_combo))
print('\nResults: ')
print('Grouped as: ', 'Benign: ', normal, ' ', 'Malicious: ', malicious)

print('Confusion matrix: ', '\tTP:', TP, 'FP:', FP, '\n', '\t\t\t', 'FN:', FN, 'TN:', TN)
print('Accuracy (%): ', accuracy, '\nPrecision: ', precision, '\nRecall: ', recall, '\nF-score: ', fscore)

print('\nLegend:','\nTP: True Positive (Benign data classified as benign)', '\nFP: False Positive (Benign data classified as malicious)')
print('\nFN: False Negative (Malicious data classified as benign)', '\nTN: True Negative (Malicious data classified as malicious)')


Benign length:  4381   Gafgyt length:  1454

Results: 
Grouped as:  Benign:  3791   Malicious:  2044
Confusion matrix:  	TP: 3137 FP: 1244 
 			 FN: 654 TN: 800
Accuracy (%):  0.6747215081405312 
Precision:  0.7160465647112532 
Recall:  0.8274861514112372 
F-score:  0.7677435144395497

Legend: 
TP: True Positive (Benign data classified as benign) 
FP: False Positive (Benign data classified as malicious)

FN: False Negative (Malicious data classified as benign) 
TN: True Negative (Malicious data classified as malicious)


## All 5 algorithms

In [ ]:
# Define "classifiers" to be used
algorithms = {
    "DBSCAN - Density-Based Spatial Clustering of Applications with Noise": DBSCAN(eps=4, min_samples=1300),
    "Isolation Forest": IsolationForest(random_state=0), 
    "Local Outlier Factor": LocalOutlierFactor(n_neighbors=20, contamination=0.1),
    "EllipticEnvelope": EllipticEnvelope(support_fraction=1., contamination=0.25),
    "OCSVM": OneClassSVM(nu=0.25, gamma=0.35)}

In [ ]:
for i, (alg_name, alg) in enumerate(algorithms.items()):
    alg.fit(data_st)
    ## show some results and evaluate them
    